# Explanation:

## Score: 0.XXXX
The target variable is: 

amount_new_house_transactions: The total monetary value of new house transactions in 10,000 yuan.

We are using some of the features found in the different csv's to predict the final amount
The method used is XGBoost with cross validation

# 1. SETUP - Dependencies, dataloading

## 1.1 Dependencies and plotting style

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import sklearn
import xgboost as xgb
import optuna

from dateutil.relativedelta import relativedelta
from xgboost.callback import EarlyStopping
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pathlib import Path
from datetime import datetime

In [ ]:
# Set global plot styles
plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

## 1.2 Loads all necessary data without any processing

We first define the datapath

In [ ]:
# Define the data path
DATA_PATH = Path("/Users/nikola/Python/KaggleCompetition/data")

We load the complemetary datasets which will include many of the features

In [ ]:
# Transaction Data (Monthly)
new_house = pd.read_csv(DATA_PATH / "train" / "new_house_transactions.csv")
pre_owned = pd.read_csv(DATA_PATH / "train" / "pre_owned_house_transactions.csv")
land = pd.read_csv(DATA_PATH / "train" / "land_transactions.csv")

# Nearby Sectors Transaction Data
new_house_nearby = pd.read_csv(DATA_PATH / "train" / "new_house_transactions_nearby_sectors.csv")
pre_owned_nearby = pd.read_csv(DATA_PATH / "train" / "pre_owned_house_transactions_nearby_sectors.csv")
land_nearby = pd.read_csv(DATA_PATH / "train" / "land_transactions_nearby_sectors.csv")

# Sector Features (Static)
sector_poi = pd.read_csv(DATA_PATH / "train" / "sector_POI.csv")

# Market Indicators (Time-series)
search_index = pd.read_csv(DATA_PATH / "train" / "city_search_index.csv")
city_indexes = pd.read_csv(DATA_PATH / "train" / "city_indexes.csv")

# Convert month columns to datetime for datasets that have it
datasets_with_month = [new_house, pre_owned, land, new_house_nearby, pre_owned_nearby, land_nearby, search_index]

for df in datasets_with_month:
    if 'month' in df.columns:
        df['month'] = pd.to_datetime(df['month'])

# Store in dictionary for easy access
datasets = {
    'new_house': new_house,
    'pre_owned': pre_owned,
    'land': land,
    'new_house_nearby': new_house_nearby,
    'pre_owned_nearby': pre_owned_nearby,
    'land_nearby': land_nearby,
    'sector_poi': sector_poi,
    'search_index': search_index,
    'city_indexes': city_indexes
}

In [ ]:
new_house.tail()

# 3. DATA & FEATURE ENGINEERING - Merging, cleaning, handling missing values


In [ ]:
# ------------------------------------------
# 3.1 Create Master Dataset
# ------------------------------------------

# Start with new_house as base (contains target)
df = new_house.copy()

# Define target variable
target_col = 'amount_new_house_transactions'

# MINIMAL feature selection - only the most essential
columns_to_merge = {
    'pre_owned': [
        'month', 'sector',
        'amount_pre_owned_house_transactions',  # Direct substitute market
        'price_pre_owned_house_transactions'    # Price signals
    ],
    'new_house_nearby': [
        'month', 'sector',
        'amount_new_house_transactions_nearby_sectors'  # Spatial spillover
    ],
}

# Merge only essential datasets
for data, name in [(pre_owned, 'pre_owned'), 
                   (new_house_nearby, 'new_house_nearby')]:
    
    cols_to_use = [col for col in columns_to_merge[name] if col in data.columns]
    df = df.merge(data[cols_to_use], on=['month', 'sector'], how='left')
    print(f"Merged {name}: added {len(cols_to_use)-2} features, shape: {df.shape}")

print(f"\nDataset shape: {df.shape}")
print(f"Base features (excluding target): {df.shape[1] - 1}")

# ------------------------------------------
# 3.2 Add Seasonal Dummy Variables
# ------------------------------------------

# Create dummy variables for peak and low months
df['is_december'] = (df['month'].dt.month == 12).astype(int)
df['is_february'] = (df['month'].dt.month == 2).astype(int)

# Verify the seasonal patterns
dec_avg = df[df['is_december'] == 1][target_col].mean()
feb_avg = df[df['is_february'] == 1][target_col].mean()
other_avg = df[(df['is_december'] == 0) & (df['is_february'] == 0)][target_col].mean()

print(f"Seasonal patterns in {target_col}:")
print(f"  December (peak): {dec_avg:,.0f}")
print(f"  February (low):  {feb_avg:,.0f}")
print(f"  Other months:    {other_avg:,.0f}")
print(f"\nAdded 2 dummy variables: is_december, is_february")
print(f"Updated dataset shape: {df.shape}")

# ==========================================
# 3.3 MINIMAL FEATURE ENGINEERING
# ==========================================

# Sort for lag calculations
df = df.sort_values(['sector', 'month']).reset_index(drop=True)

# 3. Essential time features
df['month_num'] = df['month'].dt.month
df['quarter'] = df['month'].dt.quarter
df['year'] = df['month'].dt.year

# 4. One key ratio
if 'price_pre_owned_house_transactions' in df.columns:
    df['new_to_preowned_price_ratio'] = (
        df['price_new_house_transactions'] / 
        df['price_pre_owned_house_transactions'].replace(0, np.nan)
    )

print(f"\nFinal feature count: {df.shape[1] - 3}")  # -3 for month, sector, target

# ------------------------------------------
# 3.6 Create Lagged Features (No Data Leakage)
# ------------------------------------------

print("\n" + "="*60)
print("CREATING LAGGED FEATURES FOR TIME SERIES FORECASTING")
print("="*60)

# Sort by sector and time to ensure proper lagging
df = df.sort_values(['sector', 'month']).reset_index(drop=True)

# Keep original features (we'll need them for generating features on future data)
features_to_lag = [
    'num_new_house_available_for_sale',
    'amount_pre_owned_house_transactions',
    'period_new_house_sell_through'
]

# Create 1-month lags
for feature in features_to_lag:
    if feature in df.columns:
        lag_col_name = f'{feature}_lag1'
        df[lag_col_name] = df.groupby('sector')[feature].shift(1)
        non_null = df[lag_col_name].notna().sum()
        print(f"{lag_col_name}: {non_null}/{len(df)} non-null values")

# Create sector-level historical averages (no leakage)
print(f"\nCreating sector historical features...")
df['sector_historical_mean'] = df.groupby('sector')[target_col].transform(
    lambda x: x.expanding().mean().shift(1)
)
df['sector_historical_std'] = df.groupby('sector')[target_col].transform(
    lambda x: x.expanding().std().shift(1)
)

# Create month-level historical averages across all sectors (seasonal patterns)
df['month_historical_mean'] = df.groupby(df['month'].dt.month)[target_col].transform(
    lambda x: x.expanding().mean().shift(1)
)

# Sector-month combination average (sector-specific seasonal pattern)
df['sector_month_avg'] = df.groupby(['sector', df['month'].dt.month])[target_col].transform(
    lambda x: x.expanding().mean().shift(1)
)

# Time trend per sector (how much growth/decline over time)
df['months_since_start'] = df.groupby('sector').cumcount()

print(f"sector_historical_mean: {df['sector_historical_mean'].notna().sum()}/{len(df)} non-null")
print(f"sector_historical_std: {df['sector_historical_std'].notna().sum()}/{len(df)} non-null")
print(f"month_historical_mean: {df['month_historical_mean'].notna().sum()}/{len(df)} non-null")
print(f"sector_month_avg: {df['sector_month_avg'].notna().sum()}/{len(df)} non-null")

# Keep time features
print(f"\nTime features: is_december, is_february, year, months_since_start")

# DON'T drop original features - keep them for future feature generation
print(f"\nKeeping original features for future predictions")

# Summary
print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)

all_features = [col for col in df.columns if col not in ['month', 'date', 'sector', target_col]]
print(f"Final feature count: {len(all_features)}")
print(f"Features: {all_features}")
print(f"\nRows with complete lag data: {df.dropna().shape[0]}/{len(df)}")

In [ ]:
# ------------------------------------------
# 3.7 Remove Data Leakage Columns
# ------------------------------------------

# These might directly calculate the target
potential_leakage = [
    'area_new_house_transactions',  # amount = area * price
    'total_price_per_unit_new_house_transactions',  # directly related to amount
    'price_new_house_transactions',   # directly related to amount
    'price_pre_owned_house_transactions',   # directly related to amount
    'quarter',  # might indirectly leak seasonal patterns
    'area_new_house_available_for_sale',
    'total_price_per_unit_pre_owned_house_transactions',  # directly related to amount
    'month_num',  # might indirectly leak seasonal patterns
    'num_new_house_transactions'    
    'area_per_unit_new_house_transactions',  # ADD
    'amount_new_house_transactions_nearby_sectors',  # ADD
    'new_to_preowned_price_ratio'  # ADD
]

df.drop(columns=[col for col in potential_leakage if col in df.columns], inplace=True)

print(f"Remaining columns: {len(df.columns)}")

# ------------------------------------------
# 3.8 Final Dataset Info
# ------------------------------------------

print("\n" + "="*40)
print("PREPROCESSING COMPLETE")
print("="*40)
print(f"Final dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
print(f"Target variable: {target_col}")
print(f"Number of features: {len(df.columns) - 1}")  # -1 for target

# Save processed data
output_path = Path("/Users/nikola/Python/KaggleCompetition/data")
df.to_csv(output_path / "processed_data.csv", index=False)
print(f"\nProcessed data saved to: {output_path / 'processed_data.csv'}")

# Keep a copy for modeling
df_processed = df.copy()

# ------------------------------------------
# 3.9 MAPE Competition Scoring Function
# ------------------------------------------

def custom_competition_score(y_true, y_pred):
    """
    Custom competition scoring function.
    
    Stage 1: If more than 30% of samples have absolute percentage errors > 100%, return 0
    Stage 2: Otherwise, calculate MAPE on samples with APE <= 1, scaled by fraction of valid samples
    
    Returns: Score between 0 and 1 (higher is better)
    """
    # Calculate absolute percentage errors
    ape = np.abs((y_pred - y_true) / y_true)
    
    # Stage 1: Check if more than 30% have errors > 100% (i.e., > 1.0)
    extreme_errors = np.sum(ape > 1.0) / len(ape)
    
    if extreme_errors > 0.3:
        return 0.0
    
    # Stage 2: Calculate MAPE on samples with APE <= 1
    valid_mask = ape <= 1.0
    
    if np.sum(valid_mask) == 0:
        return 0.0
    
    # D: set of valid samples (APE <= 1)
    valid_ape = ape[valid_mask]
    
    # Average MAPE of valid samples
    mape_valid = np.mean(valid_ape)
    
    # Fraction of valid samples
    fraction_valid = np.sum(valid_mask) / len(ape)
    
    # Scaled MAPE
    scaled_mape = mape_valid / fraction_valid
    
    # Final score: 1 - scaled_mape
    score = 1 - scaled_mape
    
    return max(0, score)  # Ensure non-negative

## Visualisation

In [ ]:
# ==========================================
# 3. FEATURE CORRELATION ANALYSIS
# ==========================================

# Calculate correlation matrix for all features
target_col = 'amount_new_house_transactions'
non_features = ['month', 'date', 'sector', target_col]
feature_cols = [col for col in df.columns if col not in non_features]

# Create correlation matrix
correlation_matrix = df[feature_cols + [target_col]].corr()

# ==========================================
# PLOT 1: Full Correlation Heatmap
# ==========================================
fig, ax = plt.subplots(figsize=(16, 14))
sns.heatmap(correlation_matrix, 
            annot=False,  # Set to True if you want to see values (can be crowded)
            cmap='coolwarm', 
            center=0,
            vmin=-1, 
            vmax=1,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": 0.8})
ax.set_title('Feature Correlation Matrix', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# ==========================================
# PLOT 2: High Correlation Pairs
# ==========================================

# Find highly correlated feature pairs (excluding target)
correlation_threshold = 0.8
feature_corr_matrix = df[feature_cols].corr()

# Get upper triangle to avoid duplicates
upper_triangle = np.triu(np.ones_like(feature_corr_matrix), k=1).astype(bool)
high_corr_pairs = []

for i in range(len(feature_corr_matrix.columns)):
    for j in range(i+1, len(feature_corr_matrix.columns)):
        corr_value = feature_corr_matrix.iloc[i, j]
        if abs(corr_value) >= correlation_threshold:
            high_corr_pairs.append({
                'Feature 1': feature_corr_matrix.columns[i],
                'Feature 2': feature_corr_matrix.columns[j],
                'Correlation': corr_value
            })

# Display high correlation pairs
if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('Correlation', 
                                                               key=abs, 
                                                               ascending=False)
    print("\n" + "="*70)
    print(f"HIGHLY CORRELATED FEATURE PAIRS (|correlation| >= {correlation_threshold})")
    print("="*70)
    print(high_corr_df.to_string(index=False))
    print(f"\nTotal pairs found: {len(high_corr_df)}")
    
    # Visualize high correlation pairs
    if len(high_corr_df) > 0:
        fig, ax = plt.subplots(figsize=(10, max(6, len(high_corr_df) * 0.4)))
        colors = ['red' if abs(x) > 0.95 else 'orange' for x in high_corr_df['Correlation']]
        ax.barh(range(len(high_corr_df)), high_corr_df['Correlation'], color=colors, alpha=0.7)
        ax.set_yticks(range(len(high_corr_df)))
        ax.set_yticklabels([f"{row['Feature 1'][:20]}... vs {row['Feature 2'][:20]}..." 
                            for _, row in high_corr_df.iterrows()], fontsize=8)
        ax.set_xlabel('Correlation Coefficient')
        ax.set_title(f'Highly Correlated Feature Pairs (|r| >= {correlation_threshold})')
        ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
else:
    print(f"\nNo feature pairs with |correlation| >= {correlation_threshold} found.")

# ==========================================
# PLOT 3: Top Correlations with Target
# ==========================================

# Get correlations with target variable
target_correlations = correlation_matrix[target_col].drop(target_col).sort_values(ascending=False)

# Plot top positive and negative correlations
n_top = 15
top_positive = target_correlations.head(n_top)
top_negative = target_correlations.tail(n_top)
top_features = pd.concat([top_positive, top_negative]).sort_values()

fig, ax = plt.subplots(figsize=(10, max(8, len(top_features) * 0.3)))
colors = ['green' if x > 0 else 'red' for x in top_features]
ax.barh(range(len(top_features)), top_features, color=colors, alpha=0.7)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features.index, fontsize=9)
ax.set_xlabel('Correlation with Target')
ax.set_title(f'Top {n_top} Positive and Negative Correlations with Target', fontsize=12)
ax.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

# Print correlation statistics
print("\n" + "="*70)
print("CORRELATION WITH TARGET STATISTICS")
print("="*70)
print(f"Highest positive correlation: {target_correlations.max():.4f} ({target_correlations.idxmax()})")
print(f"Highest negative correlation: {target_correlations.min():.4f} ({target_correlations.idxmin()})")
print(f"Mean absolute correlation: {target_correlations.abs().mean():.4f}")
print(f"Features with |correlation| > 0.5: {(target_correlations.abs() > 0.5).sum()}")
print(f"Features with |correlation| > 0.3: {(target_correlations.abs() > 0.3).sum()}")

# 4. PANEL DATA - CROSS VALIDATION

In [ ]:
# ==========================================
# 4.1 CROSS-VALIDATION SETUP
# ==========================================

print("="*60)
print("CROSS-VALIDATION SETUP")
print("="*60)

# Sort data by time (critical for time series)
df = df.sort_values(['month', 'sector']).reset_index(drop=True)

# Check data span
print(f"Data span: {df['month'].min()} to {df['month'].max()}")
print(f"Total months: {df['month'].nunique()}")
print(f"Total sectors: {df['sector'].nunique()}")
print(f"Total rows: {len(df)}")

# Define features and target
target_col = 'amount_new_house_transactions'
non_features = ['month', 'date', 'sector', target_col]
feature_cols = [col for col in df.columns if col not in non_features]

print(f"\nFeatures for model: {len(feature_cols)}")

# Prepare X and y
X = df[feature_cols]
y = df[target_col]

# Remove rows where target is NaN
mask = ~y.isna()
X = X[mask]
y = y[mask]
print(f"After removing NaN targets: {len(X)} rows")

# Simple TimeSeriesSplit
# For monthly data: test_size = number of months to use as test
# Let's use 6 months as test set for each fold
tscv = TimeSeriesSplit(
    n_splits=5,
    test_size=6 * df['sector'].nunique(),  # 6 months * number of sectors
    gap=0  # No gap needed for monthly data
)

# Display the splits
print("\nCross-validation splits:")
print("-" * 40)
for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
    train_months = df.iloc[train_idx]['month'].unique()
    test_months = df.iloc[test_idx]['month'].unique()
    print(f"Fold {i+1}:")
    print(f"  Train: {len(train_idx):,} rows ({len(train_months)} months)")
    print(f"  Test:  {len(test_idx):,} rows ({len(test_months)} months)")
    print(f"  Test period: {test_months.min()} to {test_months.max()}")

In [ ]:
# ==========================================
# 4.2 VISUALIZE CROSS-VALIDATION SPLITS
# ==========================================

# Since we have multiple sectors, let's aggregate by month for visualization
monthly_target = df.groupby('month')[target_col].sum().reset_index()
monthly_target = monthly_target.set_index('month').sort_index()

# Create figure
fig, axes = plt.subplots(5, 1, figsize=(15, 12), sharex=True)
fig.suptitle('Time Series Cross-Validation Splits', fontsize=14, y=1.02)

# Plot each fold
fold = 0
for train_idx, test_idx in tscv.split(X):
    # Get train/test months
    train_months = df.iloc[train_idx]['month'].unique()
    test_months = df.iloc[test_idx]['month'].unique()
    
    # Filter monthly aggregated data
    train_data = monthly_target[monthly_target.index.isin(train_months)]
    test_data = monthly_target[monthly_target.index.isin(test_months)]
    
    # Plot
    train_data.plot(ax=axes[fold], 
                    label='Training Set',
                    color='#1f77b4',
                    title=f'Fold {fold+1}: Train/Test Split')
    test_data.plot(ax=axes[fold], 
                   label='Test Set',
                   color='#ff7f0e')
    
    # Add vertical line at split point
    axes[fold].axvline(test_data.index.min(), 
                       color='black', 
                       ls='--', 
                       alpha=0.7,
                       label='Split Point')
    
    # Format
    axes[fold].set_ylabel('Total Amount\n(10k yuan)')
    axes[fold].legend(loc='upper left')
    axes[fold].grid(True, alpha=0.3)
    
    fold += 1

axes[-1].set_xlabel('Month')
plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*40)
print("SPLIT STATISTICS")
print("="*40)
for i, (train_idx, test_idx) in enumerate(tscv.split(X)):
    train_y = y.iloc[train_idx]
    test_y = y.iloc[test_idx]
    print(f"\nFold {i+1}:")
    print(f"  Train mean: {train_y.mean():,.0f}")
    print(f"  Test mean:  {test_y.mean():,.0f}")
    print(f"  Difference: {(test_y.mean() - train_y.mean())/train_y.mean()*100:+.1f}%")

# 5. MODELING - XGBoost and Visualisations

In [ ]:
# ==========================================
# 5.1 XGBOOST MODELING WITH CROSS-VALIDATION
# ==========================================

# Define features and target
target_col = 'amount_new_house_transactions'
non_features = [
    'month', 
    'date', 
    'sector', 
    'target_col',
    'num_new_house_available_for_sale',
    'amount_pre_owned_house_transactions',
    'period_new_house_sell_through'
]

feature_cols = [col for col in df.columns if col not in non_features]

print(f"Features: {len(feature_cols)}")
print(f"Target: {target_col}")

# Storage for results
fold = 0
preds = []
scores = []
models = []

for train_idx, test_idx in tscv.split(df):
    print(f"\n--- Fold {fold + 1} ---")
    
    # Split data
    train = df.iloc[train_idx]
    test = df.iloc[test_idx]
    
    # Prepare X and y
    X_train = train[feature_cols]
    y_train = train[target_col]
    X_test = test[feature_cols]
    y_test = test[target_col]
    
    # Handle NaN in target
    train_mask = ~y_train.isna()
    test_mask = ~y_test.isna()
    
    X_train = X_train[train_mask]
    y_train = y_train[train_mask]
    X_test = X_test[test_mask]
    y_test = y_test[test_mask]
    
    print(f"Train: {len(X_train)} | Test: {len(X_test)}")
    
    # XGBoost model with early_stopping_rounds in constructor
    reg = xgb.XGBRegressor(
        n_estimators=500,
        max_depth=5,
        learning_rate=0.01,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        eval_metric='mape',
        early_stopping_rounds=50  # Move it here
    )
    
    # Train
    reg.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100
    )
    
    # Predict
    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    
    # Calculate competition score using custom function
    comp_score = custom_competition_score(y_test.values, y_pred)
    scores.append(comp_score)
    models.append(reg)
    
    # Also calculate RMSE for reference
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # Diagnostic info
    ape = np.abs((y_pred - y_test.values) / y_test.values)
    extreme_pct = 100 * np.sum(ape > 1.0) / len(ape)
    valid_pct = 100 * np.sum(ape <= 1.0) / len(ape)
    
    print(f"Competition Score: {comp_score:.4f}")
    print(f"RMSE: {rmse:,.0f}")
    print(f"Samples with APE > 100%: {extreme_pct:.1f}%")
    print(f"Samples with APE ≤ 100%: {valid_pct:.1f}%")
    
    fold += 1

# Summary
print("\n" + "="*40)
print(f"Average Competition Score: {np.mean(scores):.4f}")
print(f"Std Competition Score: {np.std(scores):.4f}")
print(f"Best Competition Score: {np.max(scores):.4f}")
print(f"Worst Competition Score: {np.min(scores):.4f}")

# 6. PREDICTING THE FUTURE
Retraining on all data

To Predict the future we need an emtpy dataframe for future date ranges.

Run those dates through our feature creation code + lag creation

In [ ]:
# ==========================================
# 6. PREDICTIONS FOR SUBMISSION
# ==========================================

# ------------------------------------------
# 6.1 Load and Decode Submission Template
# ------------------------------------------

submission_template = pd.read_csv("/Users/nikola/Python/KaggleCompetition/data/sample_submission.csv")
print(f"Submission shape: {submission_template.shape}")

# Parse the ID format
submission_template['year'] = submission_template['id'].str.split(' ').str[0].astype(int)
submission_template['month_abbr'] = submission_template['id'].str.split('_').str[0].str.split(' ').str[1]
submission_template['sector'] = submission_template['id'].str.split('_').str[1]
submission_template['month'] = pd.to_datetime(
    submission_template['year'].astype(str) + ' ' + submission_template['month_abbr'],
    format='%Y %b'
)

# ------------------------------------------
# 6.2 Prepare Original Features for Future Months
# ------------------------------------------

# Create a dataframe with all sector-month combinations
future_base = submission_template[['month', 'sector']].copy()

# Get the last known values for each sector for the original features
last_values = df.groupby('sector')[['num_new_house_available_for_sale',
                                     'amount_pre_owned_house_transactions', 
                                     'period_new_house_sell_through']].last()

# Merge these values with future months (assume they stay constant)
future_base = future_base.merge(last_values, left_on='sector', right_index=True, how='left')

# For the new sector 95, fill with overall median
for col in ['num_new_house_available_for_sale', 'amount_pre_owned_house_transactions', 'period_new_house_sell_through']:
    future_base.loc[:, col] = future_base[col].fillna(df[col].median())

# ------------------------------------------
# 6.3 Create Combined Dataset
# ------------------------------------------

# Combine historical data with future data
combined = pd.concat([
    df,
    future_base
], ignore_index=True)

combined = combined.sort_values(['sector', 'month']).reset_index(drop=True)

print(f"Combined shape: {combined.shape}")

# ------------------------------------------
# 6.4 Regenerate ALL Features on Combined Dataset
# ------------------------------------------

# Time features
combined['is_december'] = (combined['month'].dt.month == 12).astype(int)
combined['is_february'] = (combined['month'].dt.month == 2).astype(int)
combined['year'] = combined['month'].dt.year

# Lag features (will use historical values for future months)
for feature in ['num_new_house_available_for_sale', 'amount_pre_owned_house_transactions', 'period_new_house_sell_through']:
    combined[f'{feature}_lag1'] = combined.groupby('sector')[feature].shift(1)

# Time series features
combined['sector_historical_mean'] = combined.groupby('sector')[target_col].transform(
    lambda x: x.expanding().mean().shift(1)
)
combined['sector_historical_std'] = combined.groupby('sector')[target_col].transform(
    lambda x: x.expanding().std().shift(1)
)
combined['month_historical_mean'] = combined.groupby(combined['month'].dt.month)[target_col].transform(
    lambda x: x.expanding().mean().shift(1)
)
combined['sector_month_avg'] = combined.groupby(['sector', combined['month'].dt.month])[target_col].transform(
    lambda x: x.expanding().mean().shift(1)
)
combined['months_since_start'] = combined.groupby('sector').cumcount()

# Define features (same as training)
target_col = 'amount_new_house_transactions'
non_features = [
    'month', 'date', 'sector', target_col,
    'num_new_house_available_for_sale',
    'amount_pre_owned_house_transactions',
    'period_new_house_sell_through'
]
feature_cols = [col for col in combined.columns if col not in non_features]

print(f"\nInitial feature count: {len(feature_cols)}")

# ------------------------------------------
# 6.5 Get Submission Data and Filter Features
# ------------------------------------------

# Get submission rows FIRST
submission_mask = combined['month'].isin(submission_template['month'])
X_submission_temp = combined[submission_mask][feature_cols]

# NOW filter features based on missing data
features_with_data = []
for col in feature_cols:
    missing_pct = X_submission_temp[col].isna().sum() / len(X_submission_temp)
    if missing_pct < 0.9:  # Keep features with <90% missing
        features_with_data.append(col)
    else:
        print(f"Removing {col}: {missing_pct*100:.1f}% missing")

feature_cols = features_with_data
print(f"\nUsing {len(feature_cols)} features after filtering")

# ------------------------------------------
# 6.6 Train Final Model
# ------------------------------------------

train_mask = combined[target_col].notna()
X_train = combined[train_mask][feature_cols].dropna()
y_train = combined[train_mask][target_col].loc[X_train.index]

print(f"\nTraining on {len(X_train)} samples")

final_model = xgb.XGBRegressor(
    n_estimators=974,
    max_depth=4,
    learning_rate=0.03397760168814681,
    subsample=0.8701413713402404,
    colsample_bytree=0.8128659290802385,
    min_child_weight=9,
    gamma=1.13656860535782,
    random_state=42,
    eval_metric='mape'
)

final_model.fit(X_train, y_train)

# ------------------------------------------
# 6.7 Make Predictions
# ------------------------------------------

# Get submission rows with filtered features
X_submission = combined[submission_mask][feature_cols].copy()

print(f"\nSubmission features - Missing values:")
print(X_submission.isna().sum())

# Fill any remaining NaN with training medians
for col in feature_cols:
    if X_submission[col].isna().any():
        X_submission.loc[:, col] = X_submission[col].fillna(X_train[col].median())

predictions = final_model.predict(X_submission)

# Clip negative predictions to 0
predictions = np.maximum(predictions, 0)

print(f"\nPredictions:")
print(f"  Min: {predictions.min():,.0f}")
print(f"  Max: {predictions.max():,.0f}")
print(f"  Mean: {predictions.mean():,.0f}")
print(f"  Non-zero: {(predictions > 0).sum()}/{len(predictions)}")

# Create submission
submission_template['amount'] = predictions
submission_template[['id', 'amount']].to_csv("/Users/nikola/Python/KaggleCompetition/output/XGBOOST/submission.csv", index=False)
print("\n✓ Submission file saved!")